In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from datasets import input_hits_TrackML, transform_TrackML
from datasets import input_blacklist_TrackML


def transform_TrackML(hits_truth, blacklist_hits):
    hits_trackML=hits_truth[np.logical_not(hits_truth.hit_id.isin(blacklist_hits.hit_id))]

    hits=hits_trackML.rename(columns={'layer_id': 'layer', 'particle_id': 'track'})
    hits.track = hits.track.where(hits.track!=0, other=-1)
    return hits


events=[]
event_prefix = 'event00000'

for num_ev in range(1000, 1100):
    
    strnum_ev=f'{num_ev:04}'
    
    event_file=event_prefix + strnum_ev
    hits_truth=input_hits_TrackML(event_file)
    blacklist=input_blacklist_TrackML(event_file)
    hits=transform_TrackML(hits_truth,blacklist)
    hits['event_id'] = num_ev
#     hits['event_id'] = str(num_ev) + '-' + hits.volume_id.astype(str)
       
    events.append(hits)

In [ ]:
events = pd.concat(events, ignore_index=True)

In [ ]:
events

In [ ]:
events[events.event_id==1049].track.nunique()

In [ ]:
from matplotlib import pyplot as plt
events.x.hist(bins=50)

In [ ]:
size_ev = events.groupby(by="event_id").size()
size_ev

In [ ]:
size_ev.hist(bins=32, log=True)

In [ ]:
size_ev.hist(bins=np.logspace(np.log10(100), np.log10(1000), 32), figsize=(16, 10)).set_xscale("log")

In [ ]:
events.groupby(by="event_id").size()

In [ ]:
events.track.nunique()

In [ ]:
events.groupby(by="event_id").track.nunique().hist(bins=50, log=True)

In [ ]:
stats = pd.concat([size_ev.rename('hits'), events.groupby(by="event_id").track.nunique().rename('tracks')], axis=1)
stats.head()

In [ ]:
stats.plot.hexbin('hits', 'tracks', bins='log', xscale='log', figsize=(16, 10), sharex=False)

In [ ]:
stats.plot.hexbin('hits', 'tracks', gridsize=32, bins='log', xscale='log', figsize=(16, 10), sharex=False)

In [ ]:
stats.plot.hexbin('hits', 'tracks', bins='log', figsize=(16, 10), sharex=False)

In [ ]:
g = sns.jointplot(
    x="hits", 
    y="tracks", 
    kind="reg",
    data=stats
);

In [ ]:
g = sns.jointplot(
    x="hits", 
    y="tracks", 
    kind="reg",
    data=stats,
    logx=True
);
g.ax_joint.set_xscale('log')